In [1]:
import numpy as np
import pandas as pd
import csv
from itertools import combinations_with_replacement
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [2]:
def pattern(orders, demand, stock, k):
    
    """
    Generate the all possible patterns for cutting a standard-sized stock material
    into pieces of specified sizes.
    
    args:
        orders: a list of specified sizes, eg. [2, 5, 4]
        
        demand: the minimum required amount for each specified size, eg. [20, 15, 30]
        
        stock:  the standard sizes of the stock materials, eg. [10, 12]
        
        k:      the maximum possible number of pieces in each pattern. For this simple
                example the longest stock is 12 and the shortest piece is 2, so a
                12-inch stock can be cut into six 2-inch pieces.
    
    Returns:
        dfp: a dataframe represents the patterns and their scrap
             rows: all patterns
             cols: 'pattern', '1st piece', '2nd piece', ..., 'kth piece', 'scrap'
             
        dfa: a dataframe represents the number of times order i is appeard in pattern j
             rows: all orders (specified size pieces)
             cols: 'order', pattern', 'count'
        
        dfc: a dataframe represents the scrap of pattern j
             rows: all patterns
             cols: 'pattern', 'scrap'
        
        dfd: a dataframe represents the demand for order i
             rows: all orders
             cols: 'order', 'demand'    
    """
    
    p =[]  # pattern data
    for i in range(len(stock)):        
        p1 = []  # pattern data for the current stock level
        
        for j in range(1, k+1):
            
            # generates combinations with replacement of length j from the orders list
            comb = list(combinations_with_replacement(orders, j))
            
            # pad each combination with zeros to make it of length k+1
            comb = [list(c) + [0]*(k+1-len(c)) for c in comb]
            
            # extend the inner list p1 with the generated combinations
            p1.extend(comb)
        
        p1 = np.array(p1)
        p1[:, k] = stock[i] - np.sum(p1, axis=1) # add scrap as the last column in p
        
        # append the pattern data for the current stock level to the main list p
        p.append(p1)
    
    p = np.concatenate(p)
    
    # filter out patterns with scrap values < 0 or >= the minimum order value
    p = p[(p[:, k] < min(orders)) & (p[:, k] >= 0), :] # keep the feasible patterns
   
    # create an index matrix using meshgrid for order and pattern indices
    index = np.array(np.meshgrid(range(1, len(orders)+1),
                                 range(1, len(p)+1))).T.reshape(-1, 2)
    
    # initialize a matrix a for counting occurrences of each order in each pattern
    a = np.hstack((index, np.zeros((len(index), 1))))
    
    # count the occurrences of each order in each pattern and updates the matrix a
    for i in range(len(orders)):
        a[i*len(p):(i+1)*len(p), 2] = np.sum(p == orders[i], axis=1)
    
    # create a matrix c with pattern indices and corresponding scrap values
    c = np.hstack((np.arange(1, len(p)+1).reshape(-1, 1), p[:, k].reshape(-1, 1)))
    
    # update matrix p with pattern indices and values
    p = np.hstack((np.arange(1, len(p)+1).reshape(-1, 1), p))
    
    # create matrix d with order indices and demand values
    d = np.hstack((np.arange(1, len(orders)+1).reshape(-1, 1),
                   np.array(demand).reshape(-1, 1)))
    
    # define column names for the DataFrame dfp
    cols = ['1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th',
            '12th', '13th', '14th']
    cols = ['pattern'] + cols[:k] + ['scrap']
    
    # create DataFrames with specified column names
    dfp = pd.DataFrame(p, columns=cols)
    dfa = pd.DataFrame(a, columns=['order', 'pattern', 'count'])
    dfc = pd.DataFrame(c, columns=['pattern', 'scrap'])
    dfd = pd.DataFrame(d, columns=['order', 'demand'])

    # create csv files of this dataframes if you want to use them in other programs
    # dfp.to_csv('p.csv', index=False)
    # dfa.to_csv('a.csv', index=False)
    # dfc.to_csv('c.csv', index=False)
    # dfd.to_csv('d.csv', index=False)
    
    return dfp, dfa, dfc, dfd

In [3]:
orders = [10.75, 11, 13, 14.5, 15, 19.5, 21, 22.5, 23, 24, 25, 26, 27.5, 28, 30, 32, 34, 36, 48, 68.5]
demand = [2, 2, 7, 8, 4, 2, 12, 4, 4, 10, 18, 2, 17, 2, 7, 4, 4, 6, 3, 2]
stock = [120, 144]
k = 5

# run the function to create possible patterns
dfp, dfa, dfc, dfd = pattern(orders, demand, stock, k)

In [4]:
model = pyo.ConcreteModel() # creates a concrete model using Pyomo

# create dictionaries for Obj coefficients (c), RHS values (b), and
# constraint coefficients (a) using information from the input DataFrames
b=dict(zip(dfd['order'], dfd['demand']))
a=dict(zip(zip(dfa['order'],dfa['pattern']), dfa['count']))
c1=dict(zip(dfc['pattern'], dfc['scrap']))
c2=dict(zip(range(1,len(b)+1), orders))

model.m = len(b) # number of constraints (number of orders)
model.n = len(c1) # number of variables (number of patterns)

# the indices for constraints and variables
model.I = pyo.RangeSet(model.m)
model.J = pyo.RangeSet(model.n)

# declare Pyomo parameters a, b, and c using the dictionaries created earlier
model.a = pyo.Param(model.I, model.J, initialize=a)
model.b = pyo.Param(model.I, initialize=b)
model.c1 = pyo.Param(model.J, initialize=c1)
model.c2 = pyo.Param(model.I, initialize=c2)

# declare a decision variable x for each pattern in the set J
model.x = pyo.Var(model.J, domain=pyo.NonNegativeIntegers)

# declare a decision variable y for each order in the set I
model.y = pyo.Var(model.I, domain=pyo.NonNegativeReals)

# define the obj function to minimize the sum of scrap values associated with each pattern
def obj_expression(m):
    return pyo.summation(m.c1, m.x) + pyo.summation(m.c2, m.y)
model.Obj = pyo.Objective(rule=obj_expression, sense=pyo.minimize)

# define the constraint rule for each constraint in set I
# the constraints ensure that the demand for each order is met
def ax_constraint_rule1(m, i):
    return sum(m.a[i,j] * m.x[j] for j in m.J) >= m.b[i]

# create one constraint for each member of the set I
model.AxbConstraint_1 = pyo.Constraint(model.I, rule=ax_constraint_rule1)

def ax_constraint_rule2(m, i):
    return sum(m.a[i,j] * m.x[j] for j in m.J) - m.b[i] == m.y[i]

# create one constraint for each member of the set I
model.AxbConstraint_2 = pyo.Constraint(model.I, rule=ax_constraint_rule2)

In [5]:
# create a solver instance using the GLPK solver
solver = SolverFactory('glpk')

# Set the termination limit in seconds
solver.options['tmlim'] = 60

In [6]:
# call the solve method to solve the optimization problem defined by the Pyomo model
# tee=True argument prints information about the solver's progress and results
solution = solver.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 60 --write C:\Users\MR0708~1\AppData\Local\Temp\tmpn6vegdoa.glpk.raw
 --wglp C:\Users\MR0708~1\AppData\Local\Temp\tmpdgslt0g6.glpk.glp --cpxlp
 C:\Users\MR0708~1\AppData\Local\Temp\tmp_jnk1xup.pyomo.lp
Reading problem data from 'C:\Users\MR0708~1\AppData\Local\Temp\tmp_jnk1xup.pyomo.lp'...
40 rows, 13365 columns, 110072 non-zeros
13345 integer variables, none of which are binary
149739 lines were read
Writing problem data to 'C:\Users\MR0708~1\AppData\Local\Temp\tmpdgslt0g6.glpk.glp'...
149712 lines were written
GLPK Integer Optimizer, v4.65
40 rows, 13365 columns, 110072 non-zeros
13345 integer variables, none of which are binary
Preprocessing...
40 rows, 13345 columns, 110052 non-zeros
13345 integer variables, none of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  5.000e+00  ratio =  5.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular p

In [7]:
s = [] # scraps
for i in range(1, len(c1)+1):
    if pyo.value(model.x[i]) > 0 and pyo.value(model.c1[i])>0:
        s.append(pyo.value(model.c1[i]))

e = [] # extra cuts
for i in range(1, len(b)+1):
    if pyo.value(model.y[i]) > 0:
        e.append(pyo.value(model.y[i])*orders[i])
    
# print the objective value of the optimization model
print('obj: total scrap and extra cut is', pyo.value(model.Obj))

print('     scrap: ', end='')
print(*s, sep=' + ')

print('     extra: ', end='')
print(*e, sep=' + ')

# print the total number of stock materials needed
count = np.zeros(len(stock))
for j in range(len(stock)):
    for i in range(1, len(c1)+1):
        if pyo.value(model.x[i]) > 0:
            if np.sum(dfp.iloc[i-1,1:]) == stock[j]:
                count[j] = count[j] + 1
    print('you need', int(count[j]), 'stocks of size', stock[j])

print('-'*80)

# print the details on how to cut the stock materials to satisfy demand
# with the minimum scrap and extra picies we don't need
for i in range(1, len(c1)+1):
    if pyo.value(model.x[i]) > 0:
        print('pattern', str(i).rjust(5),
               str(dfp.iloc[i-1,1:-1].values.flatten()).ljust(35),
              'scrap ->',
              '{}{}'.format('\u001b[43m' if pyo.value(model.c1[i]) > 0 else '',
                            pyo.value(model.c1[i])) + '\033[0m',
              ':', int(pyo.value(model.x[i])), np.sum(dfp.iloc[i-1,1:]), 'roll',
              '*scrap' if pyo.value(model.c1[i]) > 0 else '')

print('-'*80)

# print the orders and demand
for i in range(1, len(b)+1):
    print('order', str(i).rjust(2), ':  ',
          'cut', str(int(sum(model.a[i,j] * pyo.value(model.x[j])
                             for j in range(1,len(c1)+1)))).rjust(2),
          '>='.rjust(2), str(b[i]).rjust(2), 'demand',
          '*extra cut' if pyo.value(model.y[i]) > 0 else '' )    

obj: total scrap and extra cut is 52.0
     scrap: 1.5 + 0.5 + 0.5 + 1.0
     extra: 22.5 + 28.0
you need 13 stocks of size 120
you need 3 stocks of size 144
--------------------------------------------------------------------------------
pattern  1128 [27.5 27.5 27.5 36.   0. ]          scrap -> 1.5 : 1 120.0 roll *scrap
pattern  1160 [30. 30. 30. 30.  0.]               scrap -> 0.0 : 1 120.0 roll 
pattern  1263 [10.75 10.75 25.   25.   48.  ]     scrap -> 0.5 : 1 120.0 roll *scrap
pattern  2691 [11. 11. 25. 25. 48.]               scrap -> 0.0 : 1 120.0 roll 
pattern  4805 [13. 21. 25. 25. 36.]               scrap -> 0.0 : 5 120.0 roll 
pattern  5077 [13. 24. 24. 25. 34.]               scrap -> 0.0 : 2 120.0 roll 
pattern  6015 [14.5 22.5 27.5 27.5 28. ]          scrap -> 0.0 : 2 120.0 roll 
pattern  6093 [14.5 23.  27.5 27.5 27.5]          scrap -> 0.0 : 2 120.0 roll 
pattern  6320 [15. 15. 26. 30. 34.]               scrap -> 0.0 : 2 120.0 roll 
pattern  7307 [19.5 22.5 22.5 27.5 27.